In [1]:
import pandas as pd

import timeit
import os

from py2neo import Graph
import time 

In [2]:
method = 'neo4j'

# if you use kubernetes, you can map the CPU_Limit & MEMORY_Limit to the arguments.
# CPU_Limit = os.getenv('CPU_LIMIT')
# MEMORY_Limit = os.getenv('MEMORY_LIMIT')
CPU_Limit = '4.0'
MEMORY_Limit = '1024m'

In [17]:
experiment1_file = '/data/experiment1.csv'

In [18]:
result_columns = ['Analysis', 'CPU Limit', 'MEMORY Limit', 'Loading Time', 'DFG Calculation Time', 'Total Time']

In [19]:
def getConnection():
    return Graph('bolt://localhost:7687', auth=("neo4j", '1234'))

In [20]:
def getDFG(conn, logName=''):
    if logName!='':
        CMD="""
            match (t1:Attribute {{key:'concept_name'}})<--(:Event {{log_concept_name:'{0}'}})-[n]->(:Event {{log_concept_name:'{0}'}})-->(t2:Attribute {{key:'concept_name'}})
            return t1.val as dfg_from, t2.val as dfg_to, count(n) as dfg_freq
        """.format(logName)
    else:
        CMD="""
        match (t1:Attribute {key:'concept_name'})<--(:Event)-[n]->(:Event)-->(t2:Attribute {key:'concept_name'})
        return t1.val as dfg_from, t2.val as dfg_to, count(n) as dfg_freq
        """
    res = conn.run(CMD)
    return res.data()

In [21]:
def getDFGFromNeo4j(result, method, CPU_Limit, MEMORY_Limit):
    conn = getConnection()

    time_start = timeit.default_timer()
    res = getDFG(conn, 'BPI2016')
    time_stop = timeit.default_timer()

    Loading_Time = 0
    DFG_Calculation_Time = time_stop-time_start
    Total_Time = time_stop-time_start

    new_row = pd.DataFrame([[method, CPU_Limit, MEMORY_Limit, Loading_Time, DFG_Calculation_Time, Total_Time]]
                           , columns=result_columns)

    result = pd.concat([result, new_row])
    
    return result

In [22]:
if os.path.isfile(experiment1_file):
    result = pd.read_csv(experiment1_file)
else:
    result = pd.DataFrame(columns=result_columns)

In [9]:
result = getDFGFromNeo4j(result, method, CPU_Limit, MEMORY_Limit)

In [10]:
result = result.sort_values(by=result_columns)
result.to_csv(experiment1_file, index = False)